In [ ]:

import pandas as pd
from bs4 import BeautifulSoup
import re
from nltk.stem import PorterStemmer
porter = PorterStemmer()
# Importing all required packages 
import nltk
from nltk.corpus import wordnet
nltk.download('wordnet')
import pprint

In [ ]:
validate_dt=open("data/validate.data","r")


In [ ]:
dict_parse=BeautifulSoup(dict_dt,'xml')

In [ ]:
# create a method returns the first synset of a given word
def naive_disambiguation(word: str):
    synsets = wordnet.synsets(word)
    sense = synsets[0]
    return sense

In [ ]:
word = 'java' # you can replae this here
sense = naive_disambiguation(word)
print('Definition:', sense.definition())
print('Examples:')
pprint.pprint(sense.examples())

In [ ]:
for synset in wordnet.synsets('java'):
    print(synset.definition())

In [ ]:
dictt_gloss={}
for i in dict_parse.find_all("lexelt"): 
    x=(i.get('item'))
    for j in i.find_all("sense"):        
        z = j.get('id')
        y=(j.get('gloss'))
        print(x,y)
        dictt_gloss.setdefault(x,[]).append(y)

print(dictt_gloss)

In [ ]:
infile1 = open("data/dictionary.xml","r")
train1 = open("data/train.data","r")
test1 = open("data//test.data","r")
validate1 = open("data/validate.data","r")

In [ ]:
def clean_up(inp):
    
    ad_s=('i','my','me','mine','said','said','also','say','u.s.','nt','much','one','m.r','m.r.','mr.','mrs.','mrs','have') # creating custom stop words to be removed
    stopwords=nltk.corpus.stopwords.words('english')
    stopwords.append(ad_s)
    
    clean_dt=[]
    for f in str(inp).split(" "):
        w=re.sub(r'[^a-zA-Z\.\%%]','', f)
        w=w.lower()
        if w not in stopwords:
            if w != '':
                w = porter.stem(w)
                clean_dt.append(w)
    return clean_dt

In [ ]:
# Reading list with all the training data
nw_train1=train1
train_List1 = [line.split('\n') for line in nw_train1]
print( train_List1)

In [ ]:
soup = BeautifulSoup(infile,'xml')

In [ ]:
dict_gloss=dict()
for i in soup.find_all("lexelt"): 
    x=(i.get('item'))
    for j in i.find_all("sense"):  
        z = j.get('id')
        y=(j.get('gloss'))
        dict_gloss.setdefault(x,[]).append(y) 

In [ ]:
# get data list of words in dataframe
dfp = pd.DataFrame(columns=['wrd','only_wrd','sense'])
cnt=0
for each_entry in train_List1:
    w,ws=get_trainWord(each_entry) 
    given=w+"."+str(ws)
    
    dfp.loc[cnt]=given,w,ws
    cnt+=1

In [ ]:
temp=(dfp[['wrd','only_wrd']].groupby(['wrd','only_wrd'])['only_wrd'].count().reset_index(name='wordSense_count'))
temp_sens=pd.DataFrame(temp)
temp_sens

In [ ]:
temp2=(dfp[['only_wrd']].groupby(['only_wrd'])['only_wrd'].count().reset_index(name='wrd_count'))
temp_wrd=pd.DataFrame(temp2)
temp_wrd

In [ ]:
for i in range(0,len(temp_sens)):
    wrd_s=temp_sens.iloc[i][0]
    #print("Sense: ",wrd_s)
    wrd=temp_sens.iloc[i][1]
    #print("Wrd: ",wrd)
    numer=temp_sens.iloc[i][2]
    #print("senseCount: ",numer)
    deno=temp_wrd.loc[temp_wrd['only_wrd']==wrd]['wrd_count'].values[0]
    #print("wrdCount: ",deno)
    pr=numer/deno
    #print(pr)
    temp_prob.loc[i]=wrd_s,pr
temp_prob

In [ ]:
def get_featureWord(word,sent):
    #print("get_featureWord : ",word,"\n",sent)
    wrd_array=[]
    if(word in sent):
        wr_index=sent.index(word)

        iter_lst=[wr_index-2,wr_index-3,wr_index-4,wr_index-5,wr_index-6,wr_index+2,wr_index+3,wr_index+4,wr_index+5,wr_index+6]
        not_needed=['.','%','%%','mr.','u.s.','u.s.','one','said','would','like']
        for k in iter_lst:

            try:
                gotdata = sent[k]
                if (gotdata not in not_needed) and len(gotdata)>3:
                    wrd_array.append(gotdata)
            except IndexError:
                #gotdata = 'null'
                pass
    return wrd_array

In [ ]:
#test
get_featureWord('allow',al[1])

In [ ]:
import re
#funtion to know the word of training, to look in dictionary
# Pass unclean string
def get_InputWord(inpList):
    word_pos=re.sub(r'[^a-zA-Z\.]','',(str((inpList)).split("|"))[0])
    word_sense=re.sub(r'[^\d]','',(str((inpList)).split("|"))[1])
    word_context=re.sub(r'[^a-zA-Z\.]','',(str((cl)).split("."))[0])

    return word_pos,word_sense,word_context

In [ ]:
wpos,ws,wc=get_InputWord(fst)
print(wpos,ws,wc)

In [ ]:
# create dcitionary of training words
dict_train=dict()
for i in train_List1: 
    cln=clean_up(i)
    wpos,ws,wc=get_InputnWord(i)
    ky=wpos+"."+ws
    dict_train.setdefault(ky,[]).append(cln)

In [ ]:
len(dict_train)

In [ ]:
# get fearue list of a word for all sentences in the data
def allFeatureList(ky):
    #print("allFeatureList : ",ky)
    ftr_lst=[]
    word=ky.split(".")[0]
    for j in range(0,len(dict_train[ky])):
        rel_dict=dict_train[ky][j]
        features = get_featureWord(word,rel_dict)
        for feature in features:
            if feature not in ftr_lst:
                ftr_lst.append(feature)
                
    return ftr_lst

In [ ]:
allFeatureList('allow.v.1')

In [ ]:
def getFeatureCount(ky):
    hm=0
   # print("getFeatureCount :" , ky)
    featuresHere=allFeatureList(ky)
    wrd=ky.split(".")[0]
    for w in featuresHere:
        vv=0
        for m in (dict_train[ky]):
            c=m.count(w)
            vv=vv+c
        df_test.loc[hm]=wrd,ky ,w, vv
        hm+=1 

    return df_test

In [ ]:
df_test.iloc[0:0]
dd=getFeatureCount('affect.v.1')
#ddd=dd.set_index(['wrd','feature']).groupby('feature').nlargest(5).reset_index() 
((dd.sort_values('feature_count',ascending = False)).iloc[1:6]).reset_index(drop=True)
#ddd.iloc[1:6]

In [ ]:
########### Execution step
# using the above function to get the count of feature word
###########
dfFeatureCount.iloc[0:0] # drop all rows from the data frame
df_test.iloc[0:0] # drop all rows from the data frame

df_test=pd.DataFrame(columns=['only_wrd','wrd_sense','feature','feature_count'])
dfFeatureCount=pd.DataFrame(columns=['only_wrd','wrd_sense','feature','feature_count'])

key_lst=list(dict_train.keys())

for l in range(0,len(key_lst)):
    ky=(key_lst[l])
    #print(ky)
    dd=getFeatureCount(ky)
    dd=((dd.sort_values('feature_count',ascending = False)).iloc[1:7])
    dfFeatureCount=dfFeatureCount.append(dd)

In [ ]:
dfFeatureCount


In [ ]:
newD=dfFeatureCount.reset_index(drop=True)

comData=pd.DataFrame(columns=['wrd','wrd_sns','feature','feature_count','wrdSense_count','P(f|s)'])

for i in range(0,len(newD)):
    wrd_sense=newD['wrd_sense'][i] 
    only_wrd=newD['only_wrd'][i]
    ftr=newD['feature'][i]    
    ftr_count=newD['feature_count'][i]
    wrdSense_count=temp_sens[temp_sens['wrd']==wrd_sense]['wrdSense_count'].values[0]
    pr=ftr_count/wrdSense_count
    comData.loc[i]=only_wrd,wrd_sense,ftr,ftr_count,wrdSense_count,pr
#print (comData)

In [ ]:
comData #

In [ ]:
from collections import Counter
dict_train['affect.v.1'][1]
dtSupervisedPredi=pd.DataFrame(columns=['Given','Predicted'])

key_lst=list(dict_train.keys())
sense=""
cnt=0

for l in range(0,len(key_lst)):
    
    ky=(key_lst[l])

    dtKey=dict_train[ky]
    wrd=ky.split(".")[0]
    
    #print('key-',ky)
    avlbl_ftr=list(comData[(comData['wrd']==wrd)]['feature'])
    
    for each in range(0,len(dtKey)):
        #print(each)
        
        toPredict_ftr=get_featureWord(wrd,dict_train[ky][each])
        #print("PredFtr",toPredict_ftr)
        #print("avlblFtr",avlbl_ftr)
        
        del pr_sens[:]
        pr_sens=[]
        net_pr=0
        
        #print('word-',wrd)

        for pr in toPredict_ftr:
            prob=0
            
            if pr in avlbl_ftr:               
                #select the max probability
                prob = max(comData[(comData['wrd']==wrd)& (comData['feature']==pr)]['P(f|s)'])

                #find sense for that probability and word
                sense = comData[(comData['wrd']==wrd)& (comData['feature']==pr)]['wrd_sns'].values[0]
                                
            if not sense:
                continue
            else:
                pr_sens.append(sense)

        if not pr_sens:
            pr_sens.append('<NotFound>')
            
        prediction,num_most_common=Counter(pr_sens).most_common(1)[0]
        dtSupervisedPredi.loc[cnt]=ky,prediction
        cnt +=1
        #print("Given: ",ky,"Predicted: ",prediction)

In [ ]:
# Supervised Model
dtSupervisedPredi

In [ ]:
accuracies = {}

def accuracy(df):
    res = (df['Given'] == df['Predicted']).values
    return ((res==True).sum()/float(len(res))) * 100  

accuracies['SupervisedModel'] = accuracy(dtSupervisedPredi)

print("{" + "\n".join("{}: {}".format(k, v) for k, v in accuracies.items()) + "}")